# Cannon_collect_all

This routine is collecting the data from the training set run

Author(s): Sven Buder

History:
171020: SB Create file

In [35]:
# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
import os
import sys
import glob
import pickle

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib.pyplot as plt
params = {'text.usetex': True, 'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],'font.family' : 'lmodern','font.size' : 11}   
plt.rcParams.update(params)

# Package to save multiple PDF pages in one PDF
from matplotlib.backends.backend_pdf import PdfPages

# Change Work directory (if Sven's computer)
try:
    localFilePath = '/shared-storage/buder/svn-repos/trunk/GALAH/'
    os.chdir(localFilePath)
    print('Current working directory: '+os.getcwd())
except:
    print('Could not change Path to '+localFilePath)

Current working directory: /shared-storage/buder/svn-repos/trunk/GALAH


In [ ]:
def export_fits(data_structure):
    
    t = tablele.Table()
    
    
    
    t.write(,)    

In [36]:
def create_galah_dict(wg3_wg4_setup):
    galah_dict = dict(

        # General
        sobject_id  = 112233000001234,
        galah_id    = 1234567,
        field_id    = 0,
        ra          = 0.0,
        dec         = 0.0,
        ebv         = 0.0,
        snr_c1_iraf = 0.0,
        snr_c2_iraf = 0.0,
        snr_c3_iraf = 0.0,
        snr_c4_iraf = 0.0,
        teff_guess  = 0.0,
        logg_guess  = 0.0,
        feh_guess   = 0.0,
        rv_guess    = 0.0,
        e_rv_guess  = 0.0,
        flag_guess  = 0.0,

    #     # General SME
    #     trained_on     = 0,
    #     rv_sme         = 0.0,
    #     e_rv_sme       = 0.0,
    #     Alpha_fe_sme   = 0.0,
    #     e_Alpha_fe_sme = 0.0,

        # General Cannon
        chi2_cannon       = 0.0,
        sp_label_distance = 0.0,
        flag_cannon       = 0,
        Alpha_fe_cannon   = 0.0,
        e_Alpha_fe_cannon = 0.0
        )

    #for each_method in ['sme','cannon']:
    for each_method in ['cannon']:
        for each_sp in final_stellar_parameters:
            galah_dict[each_sp+'_'+each_method] = 0.0
            galah_dict['e_'+each_sp+'_'+each_method] = 0.0

        for each_elem in final_elements:
            galah_dict[each_elem+'_abund_'+each_method]         = 0.0
            galah_dict['e_'+each_elem+'_abund_'+each_method]    = 0.0
            galah_dict['ld_'+each_elem+'_abund_'+each_method]   = 0.0
            galah_dict['flag_'+each_elem+'_abund_'+each_method] = 0
            
    return galah_dict

## The class 'trainingset', where we want to put in all important data and routines

In [37]:
def read_model_pickle(filename):
    """
    INPUT:
    
    filename : pickle filename path
    
    OUTPUT:
    pickle file structure
    
    """
    door = open(filename,'r')
    print('Will read pickle file: '+filename)
    model_data = pickle.load(door)
    return model_data

In [38]:
def compute_alpha_fe(data_structure,sme_cannon='cannon',alpha_elements=['O','Mg','Si','Ca','Ti']):
    """
    INPUT:
    
    data_structure : Structure, from which the alpha-process elements come from
                     This structure also has to have the keys 
                     'Alpha_fe_'+sme_cannon and 'e_Alpha_fe_'+sme_cannon
    sme_cannon     : 'sme' or 'cannon' which will be use as key within data_structure
                     default: 'cannon'
    alpha_elements : the array including which alpha-process elements shall be included
                     default: ['O','Mg','Si','Ca','Ti']
    
    OUTPUT:
    
    None, but the following entries will be filled:
    data_structure['Alpha_fe_'+sme_cannon] & data_structure['e_Alpha_fe_'+sme_cannon] 
    
    """
    for each_sobject_id in range(len(data_structure['sobject_id'])):

        
        combi = np.nansum(np.concatenate([(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]/data_structure['e_'+x+'_abund_'+sme_cannon][each_sobject_id]**2.)[(np.isfinite(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]) & (data_structure['flag_'+x+'_abund_'+sme_cannon][each_sobject_id] == 0))] for x in alpha_elements]))/np.nansum(np.concatenate([1./data_structure['e_'+x+'_abund_'+sme_cannon][each_sobject_id][np.isfinite(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]) & (data_structure['flag_'+x+'_abund_'+sme_cannon][each_sobject_id] == 0)]**2. for x in alpha_elements]))
        e_combi = np.sqrt(1./np.nansum(np.concatenate([1./(data_structure['e_'+x+'_abund_'+sme_cannon][each_sobject_id])[np.isfinite(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]) & (data_structure['flag_'+x+'_abund_'+sme_cannon][each_sobject_id] == 0)]**2. for x in alpha_elements])))

        if np.isfinite(combi):
            data_structure['Alpha_fe_'+sme_cannon][each_sobject_id] = combi
            data_structure['e_Alpha_fe_'+sme_cannon][each_sobject_id] = e_combi

In [85]:
class testset(object):
    """ 
    Trainingset is the major class for the important information on the trainingset.
    
    In the future, this should be executable without the trainingset class
    
    Initialise with:
    obs_date      : Date of observation for the particular testset
    wg3_wg4_setup : setup dictionary with all relevant data for IRAF/SME/CANNON
    
    """
    
    def __init__(self, obs_date, wg3_wg4_setup):
        os.chdir(localFilePath)
        self.obs_date           = obs_date
        if not hasattr(self,'galah_dict'):
            self.galah_dict = create_galah_dict(wg3_wg4_setup)
        self.version_cannon     = wg3_wg4_setup['version_cannon']
        self.setup_cannon       = wg3_wg4_setup['setup_cannon']
        self.nr_ccds_cannon     = wg3_wg4_setup['nr_ccds_cannon']
        self.version_reduction  = wg3_wg4_setup['version_reduction']
        self.reduction_DR       = wg3_wg4_setup['reduction_DR']
        self.version_sme        = wg3_wg4_setup['version_sme']
        self.stellar_parameters = wg3_wg4_setup['stellar_parameters']
        self.elements           = wg3_wg4_setup['elements']
        
        os.chdir('CANNON/'+self.reduction_DR+'/'+self.version_cannon)
        print('You are now working in:'+os.getcwd())
        self.SP_data = 'pickle_test/'+self.version_cannon+'_Sp_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_'+str(self.obs_date)+'_tags.pickle'
        print(self.SP_data)

        self.AB_data  = {}
        if self.elements!='None':
            for each_element in self.elements:
                try:
                    found_setup = glob.glob('pickle_test/'+self.version_cannon+'_'+each_element+'_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_'+str(self.obs_date)+'_tags.pickle')
                    self.AB_data[each_element] = found_setup[-1]
                    print('Found automatic setup configuration for '+each_element+': '+found_setup[-1])
                except:
                    print('There is no setup configuration for '+each_element)
        self.cannon_trainingset_filename = 'sobject_iraf_'+self.version_cannon+'_trainingset.fits'
        self.cannon_filename = 'sobject_iraf_'+self.version_cannon+'.fits'

    def initiate_cannon_structure(self,nstars):
        if not hasattr(self,'cannon'):
            print('cannon structure does not exist yet - initialising cannon structure now')
            self.cannon = {}
            for each_key in galah_dict.keys():
                self.cannon[each_key] = np.array([galah_dict[each_key] for x in range(nstars)])
        else: 
            print('You want to initialise cannon structure, but it already exists!')
        
    def get_SME(self):
        found_SME = glob.glob('trainingset/'+self.version_cannon+'_Sp_*_trainingset.fits')
        print('Found automatic setup configuration for SME training set:')
        print(found_SME[-1])
        self.SME=pyfits.getdata(found_SME[-1])

    def get_SP(self):
        """
        INPUT:
        None
        
        OUTPUT:
        Adds all important information from the stellar parameter run of the Cannon to the class
        """
        params, e_params, covs, chi2, sobject_id, chi2good, chi2each = read_model_pickle(self.SP_data)
        
        if not hasattr(self,'cannon'):
            self.initiate_cannon_structure(len(sobject_id))
        
        for index, each_param in enumerate(final_stellar_parameters):
            self.cannon[each_param+'_cannon'] = params[:,index]
            self.cannon['e_'+each_param+'_cannon'] = e_params[:,index]
        self.cannon['chi2_cannon'] = chi2good
        self.cannon['sobject_id'] = np.array([int(sobject_id[x]) for x in range(len(sobject_id))])
                
    def get_AB(self):
        """
        INPUT:
        None, but class attribute 'cannon' must exist!
        
        OUTPUT:
        Adds all important information from the element abundance run of the Cannon to the class
        """
        if hasattr(self,'cannon'):
            for each_element in self.elements:
                try:
                    params, e_params, covs, chi2, sobject_ids, chi2good, chi2each = read_model_pickle(self.AB_data[each_element])
                    self.cannon[each_element+'_abund_cannon'] = params[:,-1]
                    self.cannon['e_'+each_element+'_abund_cannon'] = e_params[:,-1]
                    # Here is the place to build in the function to compare with trainingset[each_element] for label distance
                except:
                    print('Element '+each_element+' not available!')
                    self.cannon[each_element+'_abund_cannon'][:] = np.nan
                    self.cannon['e_'+each_element+'_abund_cannon'][:] = np.nan
                    self.cannon['flag_'+each_element+'_abund_cannon'][:] = -1
                    self.cannon['ld_'+each_element+'_abund_cannon'][:] = np.nan
                    pass
            compute_alpha_fe(self.cannon,sme_cannon='cannon',alpha_elements=['O','Mg','Si','Ca','Ti'])
        else:
            print('You did not call testset.get_SP() yet, which initialises the CANNON attribute to save the abundances in!')
            
    def get_IRAF(self):
        if hasattr(self,'cannon'):
            print('Reading in IRAF reduction version: '+self.version_reduction)
            iraf = pyfits.getdata(localFilePath+'/DATA/'+self.version_reduction+'.fits',ext=1)
            for cannon_index,each_sobject_id in enumerate(self.cannon['sobject_id']):
                iraf_equivalent = np.where(each_sobject_id == iraf['sobject_id'])[0]
                if len(iraf_equivalent) > 0:
                    for each_iraf_label in [
                            'galah_id','field_id','ra','dec','ebv',
                            'snr_c1_iraf','snr_c2_iraf','snr_c3_iraf','snr_c4_iraf',
                            'teff_guess','logg_guess','feh_guess','rv_guess','e_rv_guess','flag_guess'
                            ]:
                        self.cannon[each_iraf_label][cannon_index] = iraf[each_iraf_label][iraf_equivalent[0]]
                else:
                    print('Something went wrong here, because there is no IRAF entry for: '+str(each_sobject_id))
        else:
            print('You did not call testset.get_SP() yet, which initialises the CANNON attribute to save the abundances in!')



In [86]:
if sys.argv[1] == '-f':
    print('You are running the Cannon in IPYNB mode')
    version_cannon = 'Cannon3.0.1'
    reduction_DR   = 'dr5.2'
    obs_date       = 131119
    print(version_cannon,reduction_DR,obs_date)
else:
    print('You are running the Cannon in PY mode')
    version_cannon = sys.argv[1]
    reduction_DR   = sys.argv[2]
    obs_date       = sys.argv[3]

wg3_wg4_setup = dict(
    version_reduction  = 'sobject_iraf_52_171009',
    reduction_DR       = reduction_DR,
    version_sme        = 'SME360_DR2_SVN331',
    version_cannon     = version_cannon,
    setup_cannon       = 'SMEmasks_it1',
    nr_ccds_cannon     = 4,
    stellar_parameters = ['Teff', 'Logg', 'Feh', 'Vmic', 'Vsini', 'Ak'],
    elements           = [
                         'Li',  'C',  'O', 'Na', 'Mg', 'Al', 'Si',  'K', 'Ca', 'Sc', 
                         'Ti',  'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Zn', 'Rb', 'Sr',
                          'Y', 'Zr', 'Mo', 'Ru', 'Ba', 'La', 'Ce', 'Nd', 'Sm', 'Eu'
                         ]
    )

You are running the Cannon in IPYNB mode
Cannon3.0.1 dr5.2 131119


Now lets create the class for the testset!

In [87]:
testset_131119 = testset(obs_date, wg3_wg4_setup)

You are now working in:/shared-storage/buder/svn-repos/trunk/GALAH/CANNON/dr5.2/Cannon3.0.1
pickle_test/Cannon3.0.1_Sp_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Li: pickle_test/Cannon3.0.1_Li_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for C: pickle_test/Cannon3.0.1_C_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for O: pickle_test/Cannon3.0.1_O_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Na: pickle_test/Cannon3.0.1_Na_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Mg: pickle_test/Cannon3.0.1_Mg_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Al: pickle_test/Cannon3.0.1_Al_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Si: pickle_test/Cannon3.0.1_Si_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration 

In [88]:
testset_131119.get_SP()

Will read pickle file: pickle_test/Cannon3.0.1_Sp_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
cannon structure does not exist yet - initialising cannon structure now


In [89]:
testset_131119.get_IRAF()

Reading in IRAF reduction version: sobject_iraf_52_171009


In [90]:
testset_131119.get_AB()

Will read pickle file: pickle_test/Cannon3.0.1_Li_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_C_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_O_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_Na_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_Mg_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_Al_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_Si_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_K_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_Ca_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_Sc_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Will read pickle file: pickle_test/Cannon3.0.1_Ti_SMEmasks_it1_

In [91]:
testset_131119.cannon

{'Ak_cannon': array([ 0.08490672,  0.08217921,  0.03461013,  0.10894264,  0.10662911,
        -0.02480637,  0.17708251,  0.06827654,  0.05395373,  0.07075526,
        -0.01120417, -0.00523701, -0.06653689,  0.01571647,  0.00907986,
         0.06664804,  0.01009439,  0.07616698,  0.09895474,  0.13979879,
         0.11047339,  0.29607206,  0.09411507,  0.28539121, -0.03498651,
        -0.01369772,  0.00090802,  0.15013721,  0.04349935, -0.03663325,
         0.00908393,  0.04070633,  0.23088536,  0.02927049,  0.13461377,
        -0.03163932,  0.18264844,  0.01244901,  0.09094454,  0.13103849,
         0.06415039, -0.06216081,  0.0399659 ,  0.17266522, -0.035949  ,
        -0.00135031, -0.01452372, -0.05906803, -0.01802885, -0.03277161,
        -0.02619586, -0.00258665,  0.05545305,  0.03810771, -0.04964108,
        -0.01288881,  0.08711223, -0.03041753, -0.04519872,  0.12294931,
         0.25521335, -0.00782147, -0.08272628, -0.04429368,  0.28261876,
         0.09581947,  0.00243144,  0.0

In [60]:
#This we do later!

# Execution

First initialise the trainingset class

In [10]:
ts = trainingset(elements=['Li','O','Mg'])

Found automatic setup configuration         CANNON/dr5.2/Cannon3.0.1/Cannon3.0.1_Sp_SMEmasks_it1_model.pickle
Found automatic setup configuration for Li: CANNON/dr5.2/Cannon3.0.1/Cannon3.0.1_Li_SMEmasks_model.pickle
Found automatic setup configuration for O: CANNON/dr5.2/Cannon3.0.1/Cannon3.0.1_O_SMEmasks_model.pickle
There is not setup configuration for Mg


First get the training set input from SME

In [11]:
ts.get_SME()

Found automatic setup configuration for SME training set:
CANNON/dr5.2/Cannon3.0.1/trainingset/Cannon3.0.1_Sp_SMEmasks_it1_trainingset.fits


Now fill the class with the results from the Stellar Parameter run

In [180]:
ts.get_SP()

Will read pickle file: CANNON/dr5.2/Cannon3.0.1/Cannon3.0.1_Sp_SMEmasks_model.pickle


In [125]:
ts.SP_data[0]

array([[[  4.71594000e+03,   8.11553675e-01,   3.05391841e-03],
        [  4.71594000e+03,   8.01937200e-01,   9.01935259e-03],
        [  4.71594000e+03,   9.99759564e-01,   1.02814758e-02],
        ..., 
        [  4.71594000e+03,   9.99352049e-01,   2.27831669e-02],
        [  4.71594000e+03,   9.01827200e-01,   3.26022158e-02],
        [  4.71594000e+03,   7.46208839e-01,   2.91229859e-02]],

       [[  4.71598600e+03,   8.78747662e-01,   3.17599870e-03],
        [  4.71598600e+03,   8.78651615e-01,   9.44230337e-03],
        [  4.71598600e+03,   1.00358389e+00,   1.02931196e-02],
        ..., 
        [  4.71598600e+03,   1.03598946e+00,   2.30715658e-02],
        [  4.71598600e+03,   9.43354829e-01,   3.30233288e-02],
        [  4.71598600e+03,   8.12003874e-01,   3.00395143e-02]],

       [[  4.71603200e+03,   9.26898791e-01,   3.26084256e-03],
        [  4.71603200e+03,   9.28397448e-01,   9.68805637e-03],
        [  4.71603200e+03,   9.99205850e-01,   1.02926495e-02],
        

In [85]:
ts.get_AB()

AttributeError: 'trainingset' object has no attribute 'AB_model'

In [13]:
def cannon_class_to_fits(full_or_trainingset = 'trainingset'):
    
    door = open('CANNON/'+ts.reduction_DR+'/'+tsoutput+'/final_output/README')
    fits_keys,fits_description = np.genfromtxt(door,usecols=(0,1), unpack=True)
    
    return(fits_keys,fits_description)
    
#     t = table.Table()
    
#     if full_or_trainingset == 'trainingset':
#         print('Here you want to start the creation of of a trainingset fits file')
#     else:
#         print('Here you want to do the full Cannon set later!')

#     for each in fits_order[:,1]:
#         if len(fits_order[:,1]) != len(cannon_ts.keys()):
#             sys.exit('You have added more to the cannon_ts structure than given in the fits_order!')

#         for each in fits_order[:,1]:
#             try:
#                 t.add_column(table.Column(name=each, data=cannon_ts[each],description=header_comments[header_comments[:,0]==each,1][0]))
#             except:
#                 print(each+' not in cannon_ts')
#         t.write('Cannon3.0.1_Sp_SMEmasks_trainingset.fits',overwrite=True)

structure = cannon_class_to_fits()
print(structure)

ValueError: Some errors were detected !
    Line #11 (got 1 columns instead of 2)

In [86]:
ts.element

{}

In [ ]:
class each_day(object):

# Below is old stuff! Don's use!

In [ ]:
sys.exit()

#DR                  = 'dr5.2'  # default: 'dr5.2', this code is also compatible with 'dr5.1'
backup_DR_date      = '170523' # insert here only the last known date! By default, the code will try to use the latest
complete_DR         = True     # default: True, otherwise provide files in 'SPECTRA/FIELD/*.fits'
field               = ''       # default: not set, if complete_DR == False, set field name here

# ADDITIONAL CORRECTIONS BY WG4

telluric_correction = False #True
skyline_correction  = True  #True
renormalise         = False

# CANNON SPECIFICATIONS

include_ccd4        = True
subset              = '_SMEmasks'
mode_in             = '_'+mode
filteroff           = 0

iteration_outlier = []

iterations = glob.glob('CANNON/'+DR+'/'+output+'/'+output+mode_in+subset+'_model.pickle')

print(iterations)

if len(iterations) > 0:
    print('STARTING NEW ITERATION!')
    door=open('CANNON/'+DR+'/'+output+'/'+output+mode_in+subset+'_training_data.pickle')
    training_spectra,filters,training_label,labels,training_fits,training_galah_ids=pickle.load(door)
    door.close()
    try:
        door=open('CANNON/'+DR+'/'+output+'/'+output+mode_in+subset+'_selftest_tags.pickle')
        label_self, error_label_self,  covs_self, chi2_self, ids_self, chi2_good_self, chi2_each_self=pickle.load(door)
    except:
        door=open('CANNON/'+DR+'/'+output+'/'+output+mode_in+subset+'_selftest_tags.pickle')
        label_self, error_label_self,  covs_self, chi2_self, ids_self, chi2_good_self=pickle.load(door)
    door.close()
    
    for each in range(len(label_self[0])):
        bias = np.mean(training_label[:,each]-label_self[:,each])
        std  = np.std(training_label[:,each]-label_self[:,each])
        rms  = (np.sum([(xx-yy)**2 for xx,yy in zip(training_label[:,each],label_self[:,each])])/len(training_label[:,each]))**0.5
        sigma_outlier   = np.where(abs(training_label[:,each]-label_self[:,each]) > 2. * rms)[0]

        iteration_outlier.append(sigma_outlier)
        
        plt.figure()
        plt.plot(training_label[:,each],training_label[:,each]-label_self[:,each],'ko',ms=2,alpha=0.2)
        plt.axhline(2.*rms,ls='dashed')
        plt.axhline(-2.*rms,ls='dashed')

    iteration_outlier = np.unique(np.concatenate((iteration_outlier)))

    t = table.Table()
    ts_old = t.read('CANNON/'+DR+'/'+output+'/trainingset/'+output+mode_in+subset+'_trainingset.fits')
    print('ORIGINAL FITS HAS '+str(len(ts_old['sobject_id']))+' ENTRIES')

    subset = subset+'_it'+str(len(iterations))
    print('THIS IS ITERATION NR. '+str(len(iterations)))
    print('EXCLUDING '+str(len(iteration_outlier))+' SPECTRA ('+str("{:.1f}".format((100*len(iteration_outlier))/len(training_label)))+'\% OF '+str(len(training_label))+' SPECTRA)')

    np.savetxt('CANNON/'+DR+'/'+output+'/trainingset/'+output+mode_in+subset+'_leftout',iteration_outlier,fmt='%s')

    print(len(ts_old))
    ts_old.remove_rows(iteration_outlier)

    print(len(ts_old))

    print(str(len(ts_old['sobject_id']))+' STARS LEFT')
    ts_old.write('CANNON/'+DR+'/'+output+'/trainingset/'+output+mode_in+subset+'_trainingset.fits',overwrite=True)

include_parameters = np.array(['Teff','Logg','Feh','Vmic','Vsini']) # 'Alpha_fe',
include_auxiliary  = np.array(['Ak']) # ['Ebv']
include_abundances = np.array([])
if mode != 'Sp':
    include_abundances = np.array([mode])
#include_abundances = np.array(['O','Na','Mg','Si','Ca','Ti','Cr'])

labels = np.concatenate((include_parameters,include_auxiliary,include_abundances))

# Define the 4 CCD grids for the Cannon leaving at least 20 km/s to ab lines
x1=np.arange(4715.94,4896.00,0.046) # ab lines 4716.3 - 4892.3
x2=np.arange(5650.06,5868.25,0.055) # ab lines 5646.0 - 5867.8
x3=np.arange(6480.52,6733.92,0.064) # ab lines 6481.6 - 6733.4
x4=np.arange(7693.50,7875.55,0.074) # ab lines 7691.2 - 7838.5

include_ccd4=True#False
plot_spectra=False#True